# Tutorial and Demo

In [ ]:
# Explain the goal of the simulator and the usecase it is designed for

# Show the architecture of the simulator

# Explain the filtering steps


In [ ]:
# Tutorial on how to set up the simulation

# Definition of the environment

# Definition of the length of the filters and of the interpolation (accuracy vs performance)

# Definition of source signals and background noise to be added to the simulation

# Generation of microphone received signals

In [ ]:
# Define simulation environment with moving source and random signal
import pyroadacoustics as pyroad
import numpy as np
env = pyroad.Environment()
env.add_source(np.array([3,5,1]))
env.add_microphone_array(np.array([[0,0,1]]))

signal = env.simulate()
# Run simulation and plot signals received at microphone array
import matplotlib.pyplot as plt
plt.plot(signal[0])

# Plot spectrogram of received signal to show doppler

# Show comb filtering effect using FFT representations

# Listen to signal produced by simulator